# 0. Package 설치

In [ ]:
!pip install -r requirements.txt

# 1. Video -> Frame 전처리
### video_path 에 입력 비디오 경로
### frame_path 에 전처리되어 나올 frame 경로
### 입력받은 비디오를 center crop하여 frame별로 저장
### 홍나실 비디오 위치 : vision@/data1/imsi
### ** 이미 전처리 된 이미지를 사용하고 싶으면 아래 경로 참조
### vision@/data1/dict/articulated-animation/data

In [2]:
import os
from pathlib import Path
from tqdm import tqdm
import shutil
import cv2
import glob

In [4]:
video_path = './data/handspeech_video/*.mp4'
frame_path = './data/handspeech_frame'
frame_size = 384
SKIP_FRAME_RATE = 6

In [8]:
video_list = glob.glob(f'{video_path}', recursive=True)

In [9]:
video_list

['./data/handspeech_video/covid_Real.mp4',
 './data/handspeech_video/covid_Avatar.mp4']

In [13]:
for i, VIDEO_PATH in enumerate(tqdm(video_list)):
    if i < len(mp4_list)*0.95:
        phaze = 'train'
    else:
        phaze = 'test'
    fname = VIDEO_PATH.split('/')[-1].split('.mp4')[0]
    raw_img_dir = f'{frame_path}/{phaze}/{fname}'
    path = Path(raw_img_dir)
    path.mkdir(parents=True, exist_ok=True)
    capture = cv2.VideoCapture(VIDEO_PATH)
    SKIP_FRAME = int(capture.get(cv2.CAP_PROP_FPS) + 0.1)//SKIP_FRAME_RATE
    w = int(capture.get(cv2. CAP_PROP_FRAME_WIDTH ))
    h = int(capture.get(cv2. CAP_PROP_FRAME_HEIGHT ))
    x = int((w-h)/2)
    y = 0
    idx = 0
    n = 0
    while(capture.isOpened()):
        ret = capture.grab()
        if ret == False:
            break
        if idx % SKIP_FRAME == 0:
            ret, frame = capture.retrieve()
            cv2.imwrite(f'{raw_img_dir}/out{n:05d}.jpg', cv2.resize(frame, (frame_size,frame_size), interpolation=cv2.INTER_AREA))
            n += 1
            # do something with frame
        idx += 1
    capture.release()

100%|██████████| 2/2 [00:15<00:00,  7.98s/it]


# 2. Config 생성
### ./config/handspeech 를 참고하여 생성하되, dataset_params -> root_dir 은 위의 frame_path를 입력해줌

# 3. Main 모델 학습
### 2.에서 생성한 config path를 입력 후 실행
### 학습 중간 결과 및 checkpoint 가 ./log 밑에 생김

In [ ]:
!python run.py --config config/handspeech.yaml --device_ids 0,1,2,3,4,5,6,7

# 4. AVD 모델 학습
### 3. 에서 훈련되어 나온 checkpoint의 경로 입력

In [ ]:
!python run.py -checkpoint {PATH-TO-MODEL.pth} --config config/handspeech.yaml --device_ids 0,1,2,3,4,5,6,7 --mode train_avd